<a href="https://colab.research.google.com/github/lassomontana/check-keyword-similarity/blob/main/check_keyword_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This project is designed to process a list of keywords, calculate their semantic similarity using TF-IDF vectorisation and cosine similarity to avoid duplicate keywords.
#And then improves the output by adding columns for similar keywords and their similarity scores. In addition, it applies conditional formatting to highlight highly similar keywords directly in an Excel file.

#The primary steps include:

# 1 Reading an Excel file containing the keywords.
# 2 Calculating the similarity between keywords.
# 3 Adding columns to show the most similar keywords and their scores.
# 4 Applying conditional formatting to highlight high similarity scores in the Excel file.

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Function to read the Excel file
def read_excel(file_path):
    return pd.read_excel(file_path)

# Function to calculate similarity and add columns
def add_similarity_columns(df):
    keywords = df.iloc[:, 1].tolist()  # Use the second column for keywords

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer().fit_transform(keywords)
    vectors = vectorizer.toarray()

    # Cosine similarity calculation (Salton)
    cosine_sim_matrix = cosine_similarity(vectors)

    # Create columns for similar keywords and scores
    num_similar = 5  # Maximum number of similar keywords to display per keyword
    for n in range(1, num_similar + 1):
        df[f'Similar Keyword {n}'] = ''
        df[f'Similarity Score {n}'] = ''

    for i in range(len(keywords)):
        similar_pairs = []
        scores = []
        for j in range(len(keywords)):
            if i != j and cosine_sim_matrix[i][j] > 0:
                similar_pairs.append((keywords[j], cosine_sim_matrix[i][j]))

        # Sort pairs by descending score
        similar_pairs = sorted(similar_pairs, key=lambda x: x[1], reverse=True)

        # Add similar keywords and scores to the appropriate columns
        for n in range(min(num_similar, len(similar_pairs))):
            df.at[i, f'Similar Keyword {n+1}'] = similar_pairs[n][0]
            df.at[i, f'Similarity Score {n+1}'] = similar_pairs[n][1]

    return df

# Function to write the modified DataFrame to an Excel file
def write_excel(df, output_file_path):
    df.to_excel(output_file_path, index=False)

# Function to apply conditional formatting
def apply_conditional_formatting(file_path):
    wb = load_workbook(file_path)
    ws = wb.active
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Iterate over cells in the similarity score columns and apply formatting
    for row in ws.iter_rows(min_row=2, min_col=3, max_col=ws.max_column):
        for cell in row:
            try:
                if float(cell.value) > 0.95:
                    cell.fill = fill
            except (ValueError, TypeError):
                continue

    wb.save(file_path)

# Paths for input and output files
input_file_path = '/content/motsclessimilarity.xlsx'  # Replace with your file path (copy paste path)
output_file_path = '/content/motsclessimilarity-export.xlsx'  # Replace with your output path

# Read the Excel file
df = read_excel(input_file_path)

# Add similarity columns
df = add_similarity_columns(df)

# Write the modified DataFrame to a new Excel file
write_excel(df, output_file_path)

# Apply conditional formatting to the output Excel file
apply_conditional_formatting(output_file_path)

# Display the modified DataFrame (optional)
df
